## Исследование производительности обрезания картинок

In [44]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from os import mkdir
from os.path import join, exists
from shutil import rmtree
from time import time

### Загрузим пример

In [34]:
GT_COLUMNS = ['frame', 'id', 'bb_left', 'bb_top',
              'bb_width', 'bb_height', 'is_consider', 'class', 'visibility']
GT_TYPES = {
    'frame': int,
    'id': int,
    'bb_left': int,
    'bb_top': int,
    'bb_width': int,
    'bb_height': int,
    'is_consider': int,
    'class': int,
    'visibility': float 
}

In [56]:
NAME = 'MOT20-02'

In [57]:
annotations = pd.read_csv(
    f'/home/nick/reidentification/data/MOT20/train/{NAME}/gt/gt.txt', names=GT_COLUMNS, dtype=GT_TYPES)
annotations

,frame,id,bb_left,bb_top,bb_width,bb_height,is_consider,class,visibility
0,722,1,547,252,68,145,1,1,0.092317
1,723,1,548,252,68,145,1,1,0.092317
2,724,1,550,252,68,145,1,1,0.089339
3,725,1,552,252,68,145,1,1,0.089339
4,726,1,554,252,68,145,1,1,0.086361
...,...,...,...,...,...,...,...,...,...
202210,2778,296,2,1,326,594,0,11,1.000000
202211,2779,296,2,1,332,595,0,11,1.000000
202212,2780,296,2,1,339,597,0,11,1.000000
202213,2781,296,2,1,346,598,0,11,1.000000


In [36]:
np.random.seed(0)
test_frames = np.random.choice(annotations['frame'].unique(), 300)
test_frames

array([173,  48, 118, 193, 324, 252, 196, 360,  10, 212, 278, 243, 293,
        88,  71,  89, 397, 315, 194,  40,  88, 175,  89, 338, 166,  26,
       334,  73, 266, 405, 116, 244, 198, 336, 339, 100, 178, 244, 286,
       148, 148, 399, 424, 289, 266, 186, 128,  33,  32, 203, 245, 152,
       164, 371, 184,  29, 291, 129, 129, 421,  54, 390,  39, 245, 274,
       336, 389, 106,  43,  32, 377, 258, 322, 426,  58, 292, 359, 120,
       268,  83,  92, 385, 399, 100,  54, 397, 122, 427,  85, 204, 325,
       263,  48, 128, 132, 357, 181, 335, 144, 149, 228, 280, 208, 398,
       374, 342,  49, 306,  70, 170, 164,  96, 198,  95, 257, 370, 179,
       293, 419, 305, 350, 388,  99,  43, 369, 406, 202, 384,   1, 395,
       371,  44, 384,  24, 188, 131, 378,  99,  63, 420, 223, 124,  83,
       228, 149, 210,  51, 412, 271,  42,  59, 194,  37, 267,  87,  44,
       361,  12, 259, 308,  81,  33, 183, 129, 295, 276, 175,  43, 372,
       185,  78, 287, 281, 126, 259,   4,  95, 227, 364, 270, 36

In [37]:
test_objects = annotations[annotations['frame'].isin(test_frames)]
test_objects

,frame,id,bb_left,bb_top,bb_width,bb_height,is_consider,class,visibility
0,1,1,199,813,140,268,1,1,0.83643
3,4,1,206,812,140,268,1,1,0.84015
9,10,1,223,812,140,268,1,1,0.84015
11,12,1,230,811,137,270,1,1,0.83764
13,14,1,236,809,133,272,1,1,0.83150
...,...,...,...,...,...,...,...,...,...
26638,421,90,4,1,495,815,0,11,1.00000
26639,422,90,4,1,496,814,0,11,1.00000
26641,424,90,4,1,498,813,0,11,1.00000
26643,426,90,4,1,501,812,0,11,1.00000


In [53]:
def clean_tree(objects):
    if (exists('temp')):
        rmtree('temp')
    mkdir('temp')
    for id in objects['id'].unique():
        mkdir(join('temp', str(int(id))))

In [58]:
def get_image_path(frame):
    return f'/home/nick/reidentification/data/MOT20/train/{NAME}/img1/{str.zfill(str(int(frame)), 6)}.jpg'

def get_file_path(object):
    return f'temp/{int(object["id"])}/{str.zfill(str(int(object["frame"])), 6)}.jpg'

get_image_path(test_objects.iloc[0]['frame']), get_file_path(test_objects.iloc[0])

('/home/nick/reidentification/data/MOT20/train/MOT20-02/img1/000001.jpg',
 'temp/1/000001.jpg')

### PIL

In [40]:
from PIL import Image

In [42]:
def crop_pil_obj(image: Image.Image, object):
    x, y, w, h = object['bb_left'], object['bb_top'], object['bb_width'], object['bb_height']
    img = image.crop(box=(x, y , x + w, y + h))
    img.save(get_file_path(object))
    return img

def crop_pil(objects: pd.DataFrame):
    for frame in tqdm(objects['frame'].unique()):
        objects_to_crop = objects[objects['frame'] == frame].to_dict('records')
        img = Image.open(get_image_path(frame))
        for x in objects_to_crop:
            # print(x)
            crop_pil_obj(img, x)

### 

### Тестирование
меряем количество секунд, ушедших на обработку одного кадра

In [45]:
res = {
    'pil': -1
}

In [59]:
clean_tree(annotations)
start_pil = time()
crop_pil(annotations)
res['pil'] = (time() - start_pil) / 300

 24%|██▍       | 667/2782 [00:57<03:00, 11.69it/s]


KeyboardInterrupt: 

In [47]:
res['pil']

0.06823237180709839